#   T, N, Array{T,N} ?
Why do I keep seeing this pattern in type parameters? <br>
Inspired by https://www.youtube.com/watch?v=fl0g9tHeghA

## 1. Reminders about type parameters

In [1]:
struct Foo{S,T,U}
    x::S
    y::T
    z::U
end

In [2]:
a = Foo(2, 2.0, "two")

Foo{Int64,Float64,String}(2, 2.0, "two")

In [3]:
typeof(a)

Foo{Int64,Float64,String}

In [4]:
dump(a)

Foo{Int64,Float64,String}
  x: Int64 2
  y: Float64 2.0
  z: String "two"


## 2. Putting information about the type of the contents of foo allows us to dispatch on the contents:

In [5]:
import Base.*

# This doesn't work because we need the "where" syntax
*(a::Foo{S,T,U},b::Foo{S,T,U}) = Foo(a.x+b.x, a.y+b.y, a.z+b.z)

LoadError: [91mUndefVarError: S not defined[39m

In [7]:
*(a::Foo{S,T,U}, b::Foo{S,T,U}) where {S,T,U} = Foo(a.x * b.x, a.y * b.y, a.z * b.z)

* (generic function with 183 methods)

In [8]:
# This includes 
# *(a::Foo{Int64,Float64,String}, b::Foo{Int64,Float64,String}) = Foo(a.x*b.x, a.y*b.y, a.z*b.z)

# Recall that * on strings does string concatenation:

In [9]:
"ab" * "cd"

"abcd"

In [10]:
a * a

Foo{Int64,Float64,String}(4, 4.0, "twotwo")

In [11]:
dump(a * a)

Foo{Int64,Float64,String}
  x: Int64 4
  y: Float64 4.0
  z: String "twotwo"


In [12]:
b = Foo( [1 1;1 1], [1 0;0 1], 100)

Foo{Array{Int64,2},Array{Int64,2},Int64}([1 1; 1 1], [1 0; 0 1], 100)

In [13]:
b * b

Foo{Array{Int64,2},Array{Int64,2},Int64}([2 2; 2 2], [1 0; 0 1], 10000)

## 3. Putting information about how an object is used is equally valuable 

In [14]:
# Build our own pretend matrix of twos, storing only the size

struct Twos{T,N} <: AbstractArray{T,N}
   size :: NTuple{N,Int}
end

Defining the following two methods, `getindex` and `size`, is sufficient to endow the type `Twos` with array-like behavior ("array semantics"):

In [18]:
Base.getindex(A::Twos{T,N}, i::Int...) where {T,N} = 2*one(T)   

Base.size(A::Twos) = A.size

In [19]:
Twos(::Type{T}, i::Vararg{Int,N}) where {T,N} = Twos{T,N}(i)

Twos(i::Vararg{Int,N}) where {N} = Twos{Int,N}(i)

Twos

In [23]:
# primitive constructor
Twos{Int,2}((3, 4))

3×4 Twos{Int64,2}:
 2  2  2  2
 2  2  2  2
 2  2  2  2

In [24]:
# convenience constructor
Twos(Float64, 3, 4)

3×4 Twos{Float64,2}:
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0

In [25]:
# more convenient
Twos(3, 4)

3×4 Twos{Int64,2}:
 2  2  2  2
 2  2  2  2
 2  2  2  2

## 4. Reshape as an example

In [2]:
v = [1:12;]

12-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

What happens when we reshape this array?

In [3]:
A = reshape(v, 2, 6)

2×6 Array{Int64,2}:
 1  3  5  7   9  11
 2  4  6  8  10  12

In [4]:
A2 = reshape(v, 3, 4)

3×4 Array{Int64,2}:
 1  4  7  10
 2  5  8  11
 3  6  9  12

These objects share data:

In [5]:
v[5] = 100

100

In [6]:
A

2×6 Array{Int64,2}:
 1  3  100  7   9  11
 2  4    6  8  10  12

In [7]:
A2

3×4 Array{Int64,2}:
 1    4  7  10
 2  100  8  11
 3    6  9  12

In [8]:
A2[3,4] = -10

-10

In [9]:
v

12-element Array{Int64,1}:
   1
   2
   3
   4
 100
   6
   7
   8
   9
  10
  11
 -10

In [10]:
A

2×6 Array{Int64,2}:
 1  3  100  7   9   11
 2  4    6  8  10  -10

## Reshaping a range

In [12]:
r = 1:12

1:12

In [14]:
dump(r)

UnitRange{Int64}
  start: Int64 1
  stop: Int64 12


In [28]:
r.start, r.stop

(1, 25)

In [16]:
r + r

2:2:24

`r` behaves like a 1D array. Manipulating it often returns a standard array:

In [25]:
r[3]

3

In [19]:
v2 = r.^2

12-element Array{Int64,1}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100
 121
 144

Here, a *new* object was created.

In [20]:
v2[3] = 400

400

In [22]:
v2

12-element Array{Int64,1}:
   1
   4
 400
  16
  25
  36
  49
  64
  81
 100
 121
 144

In [26]:
r[3]

3

Note that `r` is immutable (cannot be modified):

In [27]:
r[3] = 5

LoadError: [91mindexing not defined for UnitRange{Int64}[39m

In [28]:
r

1:12

## Reshaping a range

What happens if we reshape a range? We would like the result to behave like an array. However, the underlying "data" doesn't exist in memory. Nonetheless, Julia can handle this:

In [31]:
r = 1:12

1:12

In [37]:
A = reshape(r, 3, 4)

3×4 Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}}:
 1  4  7  10
 2  5  8  11
 3  6  9  12

This might have created a new array object, materialized in memory:

In [38]:
A2 = reshape(collect(r), 3, 4)

3×4 Array{Int64,2}:
 1  4  7  10
 2  5  8  11
 3  6  9  12

But in fact, Julia does something sneakier: it creates an object that *behaves like an array*, but reuses the range object as the underlying substrate:

In [39]:
dump(A)

Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}}
  parent: UnitRange{Int64}
    start: Int64 1
    stop: Int64 12
  dims: Tuple{Int64,Int64}
    1: Int64 3
    2: Int64 4
  mi: Tuple{} ()


In [40]:
A.parent, A.dims

(1:12, (3, 4))

In [41]:
dump(typeof(A))

Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}} <: AbstractArray{Int64,2}
  parent::UnitRange{Int64}
  dims::Tuple{Int64,Int64}
  mi::Tuple{}


The fact that the type of `A` is a subtype of `AbstractArray{Int64, 2}` is what shows us that Julia will treat the object *as if it were a matrix of `Int64`*. Some operations work correctly, while others don't:

In [48]:
svdvals(A)

3-element Array{Float64,1}:
 25.4624     
  1.29066    
  1.56509e-15

In [47]:
@which svdvals(A)

svdvals(A::AbstractArray{T,2}) where T in Base.LinAlg at linalg/svd.jl:157

In [46]:
eigvals(A)

LoadError: [91mMethodError: no method matching eigvals(::Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}})[0m
Closest candidates are:
  eigvals(::AbstractArray{TA,2}, [91m::AbstractArray{TB,2}[39m) where {TA, TB} at linalg/eigen.jl:408
  eigvals([91m::SymTridiagonal{T}[39m) where T at linalg/tridiag.jl:187
  eigvals([91m::SymTridiagonal{T}[39m, [91m::UnitRange[39m) where T at linalg/tridiag.jl:194
  ...[39m

In [49]:
dump(A)

Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}}
  parent: UnitRange{Int64}
    start: Int64 1
    stop: Int64 12
  dims: Tuple{Int64,Int64}
    1: Int64 3
    2: Int64 4
  mi: Tuple{} ()


The remaining type parameters of a `ReshapedArray` correspond to the internal representation; in this case, `UnitRange{Int64}` refers to the fact that the underlying data is coming from the `Range` object `r`. The tuple `dims` stores the information about the size of the reshaped array that Julia uses to treat `A` as an array:

In [50]:
size(A)

(3, 4)

In [51]:
A + A

3×4 Array{Int64,2}:
 2   8  14  20
 4  10  16  22
 6  12  18  24

## MappedArrays

Suppose we need to sometimes calculate the pointwise square root of an array, but you don't know in advance where the evaluation will be. It is excessive to store an entire copy of the array with the square root already taken; also, you may wish to modify the underlying array.

In [56]:
# Pkg.add("MappedArrays")
using MappedArrays

In [57]:
M = reshape([1:12;], 3, 4)

3×4 Array{Int64,2}:
 1  4  7  10
 2  5  8  11
 3  6  9  12

Let's map the array with the square root function:

In [59]:
M2 = mappedarray(√, M)

3×4 MappedArrays.ReadonlyMappedArray{Float64,2,Array{Int64,2},Base.#sqrt}:
 1.0      2.0      2.64575  3.16228
 1.41421  2.23607  2.82843  3.31662
 1.73205  2.44949  3.0      3.4641 

In [63]:
typeof(M2) <: AbstractArray{Float64, 2}

true

Again, `M2` looks to Julia like a matrix of `Float64`, as given by the first two type parameters, but in fact, the underlying data is still the original matrix of `Int`s. When and only when an indexing operation occurs, the `sqrt` function is applied. (Note that when we display the array, an indexing operation is indeed occurring.)

We can even modify the underlying array:

In [65]:
M[1, 2] = 100
M2[1, 2] 

10.0

Here, `M2[1, 2]` is automatically "updated", since it actually accesses M[1, 2] internally at every indexing operation

However, we cannot update `M2` directly, since it is a `ReadonlyMappedArray`:

In [67]:
M2[1, 3] = 100

LoadError: [91mindexing not defined for MappedArrays.ReadonlyMappedArray{Float64,2,Array{Int64,2},Base.#sqrt}[39m

To do so requires telling Julia what the inverse function is to map back from the `MappedArray` to the original data:

In [69]:
square(x) = x^2

M3 = mappedarray((√, square), M)

3×4 MappedArrays.MappedArray{Float64,2,Array{Int64,2},Base.#sqrt,#square}:
 1.0      10.0      2.64575  3.16228
 1.41421   2.23607  2.82843  3.31662
 1.73205   2.44949  3.0      3.4641 

In [72]:
M3[1,3] = 11

11

In [73]:
M

3×4 Array{Int64,2}:
 1  100  121  10
 2    5    8  11
 3    6    9  12

## Mapping a reshaped array

What happens if we map a reshaped array?

In [74]:
r = 1:12
A = reshape(r, 3, 4)
M = mappedarray(√, A)

3×4 MappedArrays.ReadonlyMappedArray{Float64,2,Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}},Base.#sqrt}:
 1.0      2.0      2.64575  3.16228
 1.41421  2.23607  2.82843  3.31662
 1.73205  2.44949  3.0      3.4641 

Here we see from the third type parameter that the underlying data is a `ReshapedArray`.

In [75]:
M2 = mappedarray((√, square), A)

3×4 MappedArrays.MappedArray{Float64,2,Base.ReshapedArray{Int64,2,UnitRange{Int64},Tuple{}},Base.#sqrt,#square}:
 1.0      2.0      2.64575  3.16228
 1.41421  2.23607  2.82843  3.31662
 1.73205  2.44949  3.0      3.4641 

In [76]:
M2[1, 2] = 11

LoadError: [91mindexed assignment fails for a reshaped range; consider calling collect[39m

# Views

A "view" into a matrix is another good example of a Holy type:

In [77]:
A = reshape([1:12;], 3, 4)

3×4 Array{Int64,2}:
 1  4  7  10
 2  5  8  11
 3  6  9  12

In [78]:
B = view(A, 1:2, 2:4)

2×3 SubArray{Int64,2,Array{Int64,2},Tuple{UnitRange{Int64},UnitRange{Int64}},false}:
 4  7  10
 5  8  11

This behaves like an `Int`eger matrix, with underlying data from an integer matrix, and indexed by `UnitRange` objects. The final type parameter indicates whether the resulting object can indexed linearly (i.e. if it is consecutive in memory).

# Summary

Tim Holy: 
> Arrays are natural for lazy evaluation. 

The type system is used to keep track of the contortions you do on the data without actually doing it on the data -- this is lazy evaluation.